In [25]:
import polars as pl
from pathlib import Path
from scipy.stats import kendalltau

In [5]:
root_path = Path("../results/metadata/")

AttributeError: 'PosixPath' object has no attribute 'expand'

In [6]:
path_base = Path(root_path, "wordnet_full")
path_10k = Path(root_path, "wordnet_vldb_10")

In [21]:
df1=pl.read_parquet("/home/soda/rcappuzz/work/starmie/results/metadata/wordnet_vldb_10/starmie-cl_company_employees-yadl-depleted-cont.parquet")
df2=pl.read_parquet("/home/soda/rcappuzz/work/starmie/results/metadata/wordnet_vldb_10/starmie-cl_company_employees-yadl-depleted.parquet")


In [22]:
df1=df1.with_columns(idx=pl.col("join_columns").list.to_struct()).unnest("idx").with_columns(idx=pl.col("cand_table") + "_" + pl.col("field_0") + "_" + pl.col("field_1"))
df2=df2.with_columns(idx=pl.col("join_columns").list.to_struct()).unnest("idx").with_columns(idx=pl.col("cand_table") + "_" + pl.col("field_0") + "_" + pl.col("field_1"))

In [24]:
x1 = df1["idx"].to_numpy()
x2 = df2["idx"].to_numpy()

In [26]:
kendalltau(x1,x2)

SignificanceResult(statistic=1.0, pvalue=0.0)

In [62]:
def prepare_res(path, label):
    list_cont = []

    for pth in path.iterdir():
        if "cont" in  str(pth) :
            list_cont.append(pth)
    pairs = []
    for _ in list_cont:
        s = _.stem
        s_sim = s.replace("-cont", "")
        pairs.append((s, s_sim))
    res = []
    for p in pairs:
        _df1 = pl.read_parquet(Path(path, p[0] + ".parquet")).head(100)
        _df2 = pl.read_parquet(Path(path, p[1] + ".parquet")).head(100)
        df1=_df1.with_columns(idx=pl.col("join_columns").list.to_struct()).unnest("idx").with_columns(idx=pl.col("cand_table") + "_" + pl.col("field_0") + "_" + pl.col("field_1"))
        df2=_df2.with_columns(idx=pl.col("join_columns").list.to_struct()).unnest("idx").with_columns(idx=pl.col("cand_table") + "_" + pl.col("field_0") + "_" + pl.col("field_1"))
        x1 = df1["idx"].to_numpy()
        x2 = df2["idx"].to_numpy()
        
        t = kendalltau(x1,x2)

        res.append((p[0], t.statistic))

    return pl.from_records(res).with_columns(name=pl.col("column_0").str.replace("starmie-cl_", ""), dl=pl.lit("yadl_" + label)).drop("column_0")    

In [63]:
res_base = prepare_res(path_base, "base")
res_10k = prepare_res(path_base, "10k")

In [64]:
print(res_base)
print(res_10k)

shape: (7, 3)
┌──────────┬───────────────────────────────────┬───────────┐
│ column_1 ┆ name                              ┆ dl        │
│ ---      ┆ ---                               ┆ ---       │
│ f64      ┆ str                               ┆ str       │
╞══════════╪═══════════════════════════════════╪═══════════╡
│ 1.0      ┆ movies_large-yadl-depleted-cont   ┆ yadl_base │
│ 1.0      ┆ us_accidents_2021-yadl-depleted-… ┆ yadl_base │
│ 1.0      ┆ us_accidents_large-yadl-depleted… ┆ yadl_base │
│ 1.0      ┆ company_employees-yadl-depleted-… ┆ yadl_base │
│ 1.0      ┆ housing_prices-yadl-depleted-con… ┆ yadl_base │
│ 0.307071 ┆ us_county_population-yadl-deplet… ┆ yadl_base │
│ 1.0      ┆ us_elections-yadl-depleted-cont   ┆ yadl_base │
└──────────┴───────────────────────────────────┴───────────┘
shape: (7, 3)
┌──────────┬───────────────────────────────────┬──────────┐
│ column_1 ┆ name                              ┆ dl       │
│ ---      ┆ ---                               ┆ ---      │